In [48]:
import numpy as np

def generate_random_complex_matrix(n):
    real_part = np.random.rand(n, n)
    imag_part = np.random.rand(n, n)
    complex_matrix = real_part + 1j * imag_part
    return complex_matrix

# Example usage
n = 4  # change this to any size you want
matrix = generate_random_complex_matrix(n)
print(matrix)

print(matrix.shape)
A_ravel = matrix.ravel()
print(A_ravel.shape)







[[0.95092332+0.9245528j  0.09045284+0.65101348j 0.85835072+0.98300812j
  0.51200095+0.10972636j]
 [0.62693033+0.97225499j 0.57682406+0.24383219j 0.54182026+0.0283155j
  0.69438768+0.01608342j]
 [0.69649858+0.68501211j 0.83901247+0.08533031j 0.52082943+0.76689343j
  0.9802984 +0.22813526j]
 [0.52683614+0.69112008j 0.94000276+0.47262998j 0.03439113+0.49057215j
  0.68949155+0.28077809j]]
(4, 4)
(16,)


In [ ]:
import numpy as np

def convert_to_sfixed(arr):
    """
    Convert each number in a NumPy array to a string representation in sfixed(14 downto -21) format.
    For complex numbers, represents both real and imaginary parts, resulting in a 72-bit string.
    
    This format has:
    - 1 sign bit
    - 14 bits for the integer part (positions 14 down to 1)
    - 21 bits for the fractional part (positions -1 down to -21)
    - Total width of 36 bits (1 + 14 + 21) for each real or imaginary part
    
    Args:
        arr (numpy.ndarray): Input array of numbers (real or complex)
        
    Returns:
        numpy.ndarray: Array of string representations
    """
    result = np.empty(arr.shape, dtype=object)
    
    for idx, val in np.ndenumerate(arr):
        if np.iscomplex(val):
            # Handle complex numbers by converting both real and imaginary parts
            real_val = val.real
            imag_val = val.imag
            
            # Process real part
            real_sign_bit = '1' if real_val < 0 else '0'
            real_abs_val = abs(real_val)
            real_int_part = int(real_abs_val)
            real_int_bits = format(real_int_part, '014b')[-14:]  # Take last 14 bits if overflow
            
            # Convert the real fractional part
            real_frac_part = real_abs_val - real_int_part
            real_frac_bits = ''
            for _ in range(21):
                real_frac_part *= 2
                bit = '1' if real_frac_part >= 1 else '0'
                real_frac_bits += bit
                if real_frac_part >= 1:
                    real_frac_part -= 1
            
            # Process imaginary part
            imag_sign_bit = '1' if imag_val < 0 else '0'
            imag_abs_val = abs(imag_val)
            imag_int_part = int(imag_abs_val)
            imag_int_bits = format(imag_int_part, '014b')[-14:]  # Take last 14 bits if overflow
            
            # Convert the imaginary fractional part
            imag_frac_part = imag_abs_val - imag_int_part
            imag_frac_bits = ''
            for _ in range(21):
                imag_frac_part *= 2
                bit = '1' if imag_frac_part >= 1 else '0'
                imag_frac_bits += bit
                if imag_frac_part >= 1:
                    imag_frac_part -= 1
            
            # Combine all parts: real part (36 bits) followed by imaginary part (36 bits)
            result[idx] = f"{real_sign_bit}{real_int_bits}{real_frac_bits}{imag_sign_bit}{imag_int_bits}{imag_frac_bits}"
        else:
            # Handle real numbers
            sign_bit = '1' if val < 0 else '0'
            abs_val = abs(val)
            
            # Convert the integer part (14 bits)
            int_part = int(abs_val)
            int_bits = format(int_part, '014b')[-14:]  # Take last 14 bits if overflow
            
            # Convert the fractional part (21 bits)
            frac_part = abs_val - int_part
            frac_bits = ''
            for _ in range(21):
                frac_part *= 2
                bit = '1' if frac_part >= 1 else '0'
                frac_bits += bit
                if frac_part >= 1:
                    frac_part -= 1
            
            # For real numbers, set imaginary part to zero (36 bits of zeroes)
            imag_sign_bit = '0'  # Positive zero
            imag_int_bits = '0' * 14
            imag_frac_bits = '0' * 21
            
            # Combine all parts
            result[idx] = f"{sign_bit}{int_bits}{frac_bits}{imag_sign_bit}{imag_int_bits}{imag_frac_bits}"
    
    return result

A_strings = convert_to_sfixed(A_ravel)

def concatenate_string_array(string_array):
    """
    Concatenate all strings in a NumPy array into a single string.
    
    This function works with arrays of any dimension by flattening the array
    first, then joining all the string elements together.
    
    Args:
        string_array (numpy.ndarray): NumPy array containing strings
        
    Returns:
        str: A single string containing all concatenated elements
    """
    # Flatten the array to handle arrays of any dimension
    flattened_array = string_array.flatten()
    
    # Join all strings into a single string
    result = ''.join(flattened_array)
    
    return result


A_string = concatenate_string_array(A_strings)
print(len(A_string))
A_string



1152


'000000000000000111100110110111110110000000000000000111011001010111101111000000000000000000101110010011111101000000000000000101001101010100011010000000000000000110110111011110011011000000000000000111110111010011001101000000000000000100000110001001001111000000000000000000111000001011100000000000000000000101000000111111010000000000000000000111110001110010110110000000000000000100100111010101010111000000000000000001111100110101111001000000000000000100010101011010010111000000000000000000001110011111110101000000000000000101100011100001101100000000000000000000001000001111000001000000000000000101100100100110110111000000000000000101011110101110011110000000000000000110101101100100110000000000000000000000101011101100000110000000000000000100001010101010100010000000000000000110001000101001100100000000000000000111110101111010011010000000000000000001110100110011100010000000000000000100001101101111010111000000000000000101100001110110100111000000000000000111100001010010000000000000000000000011110001111

In [ ]:
import numpy as np

t = np.linspace(0 , np.pi, 30 )
print(t)